Traditionally, models in Turing are defined using the `@model` macro:

In [ ]:
using Turing

@model function gdemo(x)
    # Set priors.
    s² ~ InverseGamma(2, 3)
    m ~ Normal(0, sqrt(s²))

    # Observe each value of x.
    x .~ Normal(m, sqrt(s²))

    return nothing
end

model = gdemo([1.5, 2.0])

The `@model` macro accepts a function definition and rewrites it such that call of the function generates a `Model` struct for use by the sampler.

However, models can be constructed by hand without the use of a macro.
Taking the `gdemo` model above as an example, the macro-based definition can be implemented also (a bit less generally) with the macro-free version

In [ ]:
using DynamicPPL

# Create the model function.
function gdemo2(model, varinfo, x)
    # Assume s² has an InverseGamma distribution.
    s², varinfo = DynamicPPL.tilde_assume!!(
        model.context, InverseGamma(2, 3), @varname(s²), varinfo
    )

    # Assume m has a Normal distribution.
    m, varinfo = DynamicPPL.tilde_assume!!(
        model.context, Normal(0, sqrt(s²)), @varname(m), varinfo
    )

    # Observe each value of x[i] according to a Normal distribution.
    for i in eachindex(x)
        _retval, varinfo = DynamicPPL.tilde_observe!!(
            model.context, Normal(m, sqrt(s²)), x[i], @varname(x[i]), varinfo
        )
    end

    # The final return statement should comprise both the original return
    # value and the updated varinfo.
    return nothing, varinfo
end
gdemo2(x) = DynamicPPL.Model(gdemo2, (; x))

# Instantiate a Model object with our data variables.
model2 = gdemo2([1.5, 2.0])

We can sample from this model in the same way:

In [ ]:
chain = sample(model2, NUTS(), 1000; progress=false)

The subsequent pages in this section will show how the `@model` macro does this behind-the-scenes.